# MMG

In [1]:
import _init_paths
import tensorflow as tf
from dataset.Dummy import Dummy
from models.CRG import CRG
from models.MMG import MMG
from models.RRG import RRG
from dataset.campus import *

In [2]:
# Architecture parameters
# MMG
mmg_hidden_dim = [256, 128, 64]
mmg_output_dim = 1

In [3]:
campus_dataset = Campus('../../data/Campus/node_features.pkl', '../../data/Campus/edge_features.pkl', '../../data/Campus/GT_graphs.pkl')
node_features, adjacency, edge_features, gt_graphs = campus_dataset._generate_graphs()
adjacency_sparse= tf.sparse.from_dense(adjacency[0:400])
node_features=node_features[0:400]
edge_features=edge_features[0:400]
gt_graphs=gt_graphs[0:400]

print(adjacency.shape)
print(node_features.shape)
print(edge_features.shape)
print(gt_graphs.shape)

(1295, 9, 9)
(400, 9, 512)
(400, 9, 9, 1)
(400, 9, 9, 1)


In [7]:
# Architecture Init
mmg_model = MMG(mmg_hidden_dim, mmg_output_dim)


In [8]:
adjacency=tf.cast(adjacency,tf.float32)
node_features=tf.cast(node_features,tf.float32)
edge_features=tf.cast(edge_features,tf.float32)
gt_graphs = tf.cast(gt_graphs,tf.float32)

In [9]:
# Forward Pass
# MMG
mmg_output = mmg_model(adjacency_sparse, node_features, edge_features)
print(mmg_output[0][1])
gt_graphs[0][1]

tf.Tensor(
[0.52087945 0.51606476 0.54030573 0.5114838  0.50289243 0.53289294
 0.5168762  0.49992716 0.51531136], shape=(9,), dtype=float32)


<tf.Tensor: shape=(9, 1), dtype=float32, numpy=
array([[0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.]], dtype=float32)>

In [10]:
# train_step 
# j'ai mis la sortie en target juste pr être sur d'avoir les bnes dimensions
mmg_model.train_step([adjacency_sparse, node_features, edge_features], gt_graphs )

In [11]:
EPOCHS = 50
for epoch in range(EPOCHS):
    for k in range(0,390,10):
        a_sparse= tf.sparse.from_dense(adjacency[k:k+10])
        n_features=node_features[k:k+10]
        e_features=edge_features[k:k+10]
        g_graphs=gt_graphs[k:k+10]
        mmg_model.train_step([a_sparse, n_features, e_features], g_graphs )

In [12]:
output = mmg_model(tf.sparse.from_dense(adjacency[0:15]), node_features[0:15], edge_features[0:15])

In [13]:
output[14][1]

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.3795872e-07, 1.8044177e-07, 9.1643176e-10, 1.2438804e-02,
       9.9640262e-01, 5.0289149e-05, 5.6472123e-03, 9.9710441e-01,
       5.4354223e-05], dtype=float32)>

In [14]:
gt_graphs[14][1]

<tf.Tensor: shape=(9, 1), dtype=float32, numpy=
array([[0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.]], dtype=float32)>